In [ ]:
# data preprocessing
import tensorflow as tf
import h5py
import scipy.io
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler

export_path = '/Your export path'
f = h5py.File('TRAIN38.mat','r')
arrays = {}
for k, v in f.items():
  arrays[k] = np.array(v)
f.close()
train_data = arrays['data'].transpose()
train_region = arrays['region'].transpose()
prob_idx = arrays['prob_idx'].transpose()
del arrays, f
curr_set = np.where(prob_idx != 38)[0]
set_data = train_data[curr_set,:]
set_region = train_region[curr_set,:]
print(set_data.shape)
curr_val = np.where(prob_idx == 38)[0]
val_data = train_data[curr_val,:]
val_label = train_region[curr_val,:]
print(val_data.shape)
del curr_set, curr_val, train_data, train_region
X_train1, X_train2, y_train1, y_train2 = train_test_split(set_data,set_region,test_size = 0.01, random_state = 42)
del set_data, set_region
scaler = StandardScaler()
scaler.fit(X_train1)
X_train1 = scaler.transform(X_train1)
val_data = scaler.transform(val_data)

In [ ]:
# train model
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend as K
from keras import activations
# Model configuration
batch_size = 128
no_epochs = 25
no_classes = 102

def create_reg_model():
  input_shape = [341]
  model = Sequential()
  model.add(Dense(4096, activation='relu', input_shape=input_shape, kernel_regularizer=tf.keras.regularizers.l2(0.00001)))
  model.add(Dropout(0.5))
  model.add(Dense(4096, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00001)))
  model.add(Dropout(0.5))
  model.add(Dense(4096, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00001)))
  model.add(Dropout(0.5))
  model.add(Dense(4096, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00001)))
  model.add(Dropout(0.5))
  model.add(Dense(no_classes, activation='softmax', name='visualized_layer'))
  return model
model = create_reg_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train1,y_train1,
          epochs=no_epochs,
          batch_size = batch_size,
          validation_data=(val_data,val_label))
model.save(export_path + 'dense_4x4096_model')

In [ ]:
#apply model
filepath = 'DEMO38.mat'
arrays = {}
f = h5py.File(filepath,'r')
for k, v in f.items():
  arrays[k] = np.array(v)
f.close()
multidim_data = arrays['multidim_data'].transpose()
del arrays, f
multidim_data = scaler.transform(multidim_data)
predicted_ann = model.predict(multidim_data)
predictpath = 'Your predictpath'
scipy.io.savemat(predictpath +'dense_4x4096_model_prediction.mat', mdict={'predicted_ann':predicted_ann})

In [ ]:
# calculate and visualize saliency
!pip install https://github.com/raghakot/keras-vis/archive/master.zip

from vis.visualization import visualize_saliency
from vis.utils import utils
import matplotlib.pyplot as plt
# Numbers to visualize
indices_to_visualize = [ 10000, 20000, 30000, 40000, 50000, 60000 ] #or range(0,173383)
layer_index = utils.find_layer_idx(model, 'visualized_layer')
# Visualize
for index_to_visualize in indices_to_visualize:
  # Get input
  input_spectrum = val_data[index_to_visualize,:,:]
  input_class = np.argmax(val_label[index_to_visualize,:])
  print(str(input_class))
  # Generate visualization
  #visualize_saliency_with_losses(input_tensor, losses, seed_input, keepdims=True)
  visualization = visualize_saliency(model, layer_index, filter_indices=input_class, seed_input=input_spectrum, keepdims=True)
  plt.figure()
  plt.title(str(input_class))
  plt.plot(visualization,'g')
  plt.plot(scaler.inverse_transform(input_spectrum),alpha=0.25)
  plt.plot(input_spectrum,'b',alpha=0.3)
  plt.axvspan(0, 15, color='gray', alpha=0.3)
  plt.axvspan(225, 230, color='gray', alpha=0.2)
  plt.show